# Dependência

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
# !pip install gcloud
# !gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

# Tratamento

In [ ]:
# Load Human Resources data from Excel file
df = pd.read_excel("Base_Estadic_2023(2).xlsx", sheet_name='Recursos Humanos')

# Create two separate dataframes for different question groups (EREH01* and EREH03*)
df1 = df[['Sigla UF', 'PopUF', 'EREH0111', 'EREH0112', 'EREH0113', 'EREH0114', 'EREH0115']]
df2 = df[['Sigla UF', 'PopUF', 'EREH0311', 'EREH0312', 'EREH0313', 'EREH0314', 'EREH0315']]

# Rename columns to meaningful Portuguese names
df1 = df1.rename(columns={
    'EREH0111': 'Estatutários',
    'EREH0112': 'Celetistas',
    'EREH0113': 'Somente comissionados',
    'EREH0114': 'Estagiários',
    'EREH0115': 'Sem vínculo permanente',
    'PopUF': 'pop_projetada',
    'Sigla UF': 'sigla_uf'
})

df2 = df2.rename(columns={
    'EREH0311': 'Estatutários',
    'EREH0312': 'Celetistas',
    'EREH0313': 'Somente comissionados',
    'EREH0314': 'Estagiários',
    'EREH0315': 'Sem vínculo permanente',
    'PopUF': 'pop_projetada',
    'Sigla UF': 'sigla_uf'
})

# Transform both dataframes from wide to long format
df1 = df1.melt(
    id_vars=['sigla_uf', 'pop_projetada'],
    value_vars=['Estatutários', 'Celetistas', 'Somente comissionados', 'Estagiários', 'Sem vínculo permanente'],
    var_name='tipo_vinculo',
    value_name='quantidade_vinculos'
)

df2 = df2.melt(
    id_vars=['sigla_uf', 'pop_projetada'],
    value_vars=['Estatutários', 'Celetistas', 'Somente comissionados', 'Estagiários', 'Sem vínculo permanente'],
    var_name='tipo_vinculo',
    value_name='quantidade_vinculos'
)

# Combine both dataframes
df = pd.concat([df1, df2])

# Clean and convert numeric values
df['quantidade_vinculos'] = pd.to_numeric(df['quantidade_vinculos'], errors='coerce')
df['quantidade_vinculos'] = df['quantidade_vinculos'].fillna(0)

# Select and reorder columns
df = df[['tipo_vinculo', 'sigla_uf', 'quantidade_vinculos']]

# Aggregate data by UF and employment type
df = df.groupby(['sigla_uf', 'tipo_vinculo'], as_index=False)['quantidade_vinculos'].sum()

# Add year column and finalize column order
df['ano'] = 2023
df = df[['ano', 'tipo_vinculo', 'sigla_uf', 'quantidade_vinculos']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ano                  135 non-null    int64  
 1   tipo_vinculo         135 non-null    object 
 2   sigla_uf             135 non-null    object 
 3   quantidade_vinculos  135 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.3+ KB


# Upload

In [ ]:
# Define the BigQuery table schema with field types and descriptions
schema = [
    bigquery.SchemaField('ano', 'INTEGER', description='Ano de referencia da informacao'),
    bigquery.SchemaField('tipo_vinculo', 'STRING', description='Tipo de vinculo.'),
    bigquery.SchemaField('sigla_uf', 'STRING', description='Sigla da Unidade da Federação.'),
    bigquery.SchemaField('quantidade_vinculos', 'INTEGER', description='Quantidade de vinculos')
]

# Initialize connection to BigQuery client for the specified project
client = bigquery.Client(project='repositoriodedadosgpsp')

# Create reference to the target dataset
dataset_ref = client.dataset('perfil_remuneracao')

# Create reference to the target table following naming convention: FONTE_algo_intuitivo_dado
table_ref = dataset_ref.table('ESTADIC_quantidade_vinculos_v1')

# Configure the data upload job with the defined schema
job_config = bigquery.LoadJobConfig(schema=schema)

# Execute the job to upload DataFrame to BigQuery
job = client.load_table_from_dataframe(
    dataframe=df,
    destination=table_ref,
    job_config=job_config
)

# Wait for the job to complete
job.result()